### This is my work in progress exploring this dataset. 
#### Brian of London

In [133]:
import pandas as pd
import datetime
import csv
import plotly as py
import plotly.graph_objs as go
import plotly.express as px
import plotly.io as pio

In [134]:
df = pd.read_csv('data/nyt_ts.csv', index_col='timestamp')
states = df['state'].unique()

In [149]:
df.index = pd.to_datetime(df.index)
df.tz_convert('US/Eastern')

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c,r_to_d
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04 05:01:44-05:00,0,0,0,edison,alaska,367000,163387,300495,318608,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0,0.000,NaN
2020-11-04 02:47:41-05:00,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,...,113744.0,68815.120,40037.888,0.0,4890.992,68815.120,40037.888,0,4890.992,1.718750
2020-11-04 03:57:55-05:00,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,...,18731.0,11500.834,6499.657,0.0,730.509,81339.650,45968.825,0,5166.525,1.769452
2020-11-04 13:34:58-05:00,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,...,39556.0,24880.724,13053.480,0.0,1621.796,108207.499,56770.230,0,7053.271,1.906061
2020-11-09 14:09:13-05:00,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,...,18939.0,11780.058,6363.504,0.0,795.438,118783.340,64165.920,0,8020.740,1.851190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-11 15:35:54-05:00,9846,276630,99,edison,wyoming,276765,174419,249061,255849,0.700,...,6.0,4.200,1.596,0.0,0.204,193641.000,73583.580,0,9405.420,2.631579
2020-11-11 15:36:41-05:00,9847,276648,99,edison,wyoming,276765,174419,249061,255849,0.700,...,18.0,12.600,4.788,0.0,0.612,193653.600,73588.368,0,9406.032,2.631579
2020-11-11 15:37:31-05:00,9848,276679,99,edison,wyoming,276765,174419,249061,255849,0.700,...,31.0,21.700,8.246,0.0,1.054,193675.300,73596.614,0,9407.086,2.631579


In [150]:
df['votes_prev'] = df.groupby('state')['votes'].shift(1)
df['votes_diff'] = [v - vp for v,vp in zip(df['votes'],df['votes_prev'])]
df.fillna(0, inplace=True)
df[['state','votes','votes_diff','votes_prev']].head(50)


,state,votes,votes_diff,votes_prev
timestamp,,,,
2020-11-04 10:01:44+00:00,alaska,0,0.0,0.0
2020-11-04 07:47:41+00:00,alaska,113744,113744.0,0.0
2020-11-04 08:57:55+00:00,alaska,132475,18731.0,113744.0
2020-11-04 18:34:58+00:00,alaska,172031,39556.0,132475.0
2020-11-09 19:09:13+00:00,alaska,190970,18939.0,172031.0
2020-11-11 02:46:15+00:00,alaska,240234,49264.0,190970.0
2020-11-11 13:18:28+00:00,alaska,260983,20749.0,240234.0
2020-11-12 16:41:43+00:00,alaska,286788,25805.0,260983.0
2020-11-13 02:25:15+00:00,alaska,312033,25245.0,286788.0


In [151]:
df


,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c,r_to_d
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04 10:01:44+00:00,0,0,0,edison,alaska,367000,163387,300495,318608,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0,0.000,0.000000
2020-11-04 07:47:41+00:00,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,...,113744.0,68815.120,40037.888,0.0,4890.992,68815.120,40037.888,0,4890.992,1.718750
2020-11-04 08:57:55+00:00,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,...,18731.0,11500.834,6499.657,0.0,730.509,81339.650,45968.825,0,5166.525,1.769452
2020-11-04 18:34:58+00:00,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,...,39556.0,24880.724,13053.480,0.0,1621.796,108207.499,56770.230,0,7053.271,1.906061
2020-11-09 19:09:13+00:00,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,...,18939.0,11780.058,6363.504,0.0,795.438,118783.340,64165.920,0,8020.740,1.851190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-11 20:35:54+00:00,9846,276630,99,edison,wyoming,276765,174419,249061,255849,0.700,...,6.0,4.200,1.596,0.0,0.204,193641.000,73583.580,0,9405.420,2.631579
2020-11-11 20:36:41+00:00,9847,276648,99,edison,wyoming,276765,174419,249061,255849,0.700,...,18.0,12.600,4.788,0.0,0.612,193653.600,73588.368,0,9406.032,2.631579
2020-11-11 20:37:31+00:00,9848,276679,99,edison,wyoming,276765,174419,249061,255849,0.700,...,31.0,21.700,8.246,0.0,1.054,193675.300,73596.614,0,9407.086,2.631579


In [138]:
# Calculate absolute vote values per time stamp
df['trump_v'] = [(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes_diff'])]
df['biden_v'] = [(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes_diff'])]
df['trd_v']= [(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes_diff'])]
df['check_v'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_v'],df['biden_v'],df['trd_v'],df['votes_diff'])]

df['trump_c'] = [(sh * v) for sh,v in zip(df['vote_share_rep'],df['votes'])]
df['biden_c'] = [(sh * v) for sh,v in zip(df['vote_share_dem'],df['votes'])]
df['trd_c']= [(sh * v) for sh,v in zip(df['vote_share_trd'],df['votes'])]
df['check_c'] = [d-(a+b+c) for a,b,c,d in zip(df['trump_c'],df['biden_c'],df['trd_c'],df['votes'])]

# check_v is showing rounding errors
df.groupby('state').last()

,Unnamed: 0,votes,eevp,eevp_source,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,vote_share_dem,...,votes_prev,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c
state,,,,,,,,,,,,,,,,,,,,,
alabama,186,2309900,99,edison,2300000,1318255,2074338,2123372,0.621,0.365,...,2297216.0,12684.0,7876.764,4629.660,0.0,177.576,1434447.900,8.431135e+05,0,32338.600
alaska,12,349473,99,edison,367000,163387,300495,318608,0.532,0.430,...,339523.0,9950.0,5293.400,4278.500,0.0,378.100,185919.636,1.502734e+05,0,13279.974
arizona,506,3385845,99,edison,3400000,1252401,2306559,2604657,0.491,0.494,...,3385747.0,98.0,48.118,48.412,0.0,1.470,1662449.895,1.672607e+06,0,50787.675
arkansas,381,1217330,99,edison,1225000,684872,1069468,1130635,0.624,0.348,...,1217300.0,30.0,18.720,10.440,0.0,0.840,759613.920,4.236308e+05,0,34085.240
california,703,17205179,99,edison,17400000,4483810,13038547,14237884,0.342,0.636,...,17204856.0,323.0,110.466,205.428,0.0,7.106,5884171.218,1.094249e+07,0,378513.938
colorado,846,3256062,99,edison,3300000,1202484,2569520,2780247,0.419,0.554,...,3254941.0,1121.0,469.699,621.034,0.0,30.267,1364289.978,1.803858e+06,0,87913.674
connecticut,1074,1823932,99,edison,1850000,673215,1558993,1644920,0.392,0.592,...,1821889.0,2043.0,800.856,1209.456,0.0,32.688,714981.344,1.079768e+06,0,29182.912
delaware,1085,504010,99,edison,504010,185127,413921,443814,0.398,0.588,...,502392.0,1618.0,643.964,951.384,0.0,22.652,200595.980,2.963579e+05,0,7056.140
florida,1389,11067456,99,edison,11067456,4617886,8474179,9501617,0.512,0.479,...,11067475.0,-19.0,-9.728,-9.101,-0.0,-0.171,5666537.472,5.301311e+06,0,99607.104


In [152]:
filt = df['state'] == 'new-hampshire'
df_arizona = df[filt]
df_arizona

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c,r_to_d
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04 09:15:43+00:00,6000,0,0,edison,new-hampshire,805000,345790,710972,744296,0.000,...,0.0,0.000,0.000,0.0,0.000000e+00,0.000,0.000,0,0.000,0.000000
2020-11-03 14:16:25+00:00,6001,26,0,edison,new-hampshire,805000,345790,710972,744296,0.615,...,26.0,15.990,10.010,0.0,0.000000e+00,15.990,10.010,0,0.000,1.597403
2020-11-04 09:15:43+00:00,6002,0,0,edison,new-hampshire,805000,345790,710972,744296,0.000,...,-26.0,-0.000,-0.000,-0.0,-2.600000e+01,0.000,0.000,0,0.000,0.000000
2020-11-03 14:16:25+00:00,6003,26,0,edison,new-hampshire,805000,345790,710972,744296,0.615,...,26.0,15.990,10.010,0.0,0.000000e+00,15.990,10.010,0,0.000,1.597403
2020-11-04 00:10:45+00:00,6004,2613,0,edison,new-hampshire,805000,345790,710972,744296,0.252,...,2587.0,651.924,1935.076,0.0,0.000000e+00,658.476,1954.524,0,0.000,0.336898
2020-11-04 00:15:59+00:00,6005,5315,1,edison,new-hampshire,805000,345790,710972,744296,0.278,...,2702.0,751.156,1950.844,0.0,0.000000e+00,1477.570,3837.430,0,0.000,0.385042
2020-11-04 00:24:02+00:00,6006,7298,1,edison,new-hampshire,805000,345790,710972,744296,0.288,...,1983.0,571.104,1411.896,0.0,-2.273737e-13,2101.824,5196.176,0,0.000,0.404494
2020-11-04 00:27:18+00:00,6007,9583,1,edison,new-hampshire,805000,345790,710972,744296,0.315,...,2285.0,719.775,1565.225,0.0,0.000000e+00,3018.645,6564.355,0,0.000,0.459854
2020-11-04 00:30:09+00:00,6008,16047,2,edison,new-hampshire,805000,345790,710972,744296,0.411,...,6464.0,2656.704,3774.976,0.0,3.232000e+01,6595.317,9371.448,0,80.235,0.703767


In [140]:
# Calculate the ratio of D/R
# filt = (df['trump_v'] == 0) | (df['biden_v'] == 0)
#dfn = df[~filt]
def myDiv(r,d):
    if d == 0:
        return float('NaN')
    else:
        return r/d


dfn = df

dfn['r_to_d'] = [myDiv(r,d) for r,d in zip(dfn['trump_v'],dfn['biden_v'])]

dfn

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c,r_to_d
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04 10:01:44+00:00,0,0,0,edison,alaska,367000,163387,300495,318608,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0,0.000,NaN
2020-11-04 07:47:41+00:00,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,...,113744.0,68815.120,40037.888,0.0,4890.992,68815.120,40037.888,0,4890.992,1.718750
2020-11-04 08:57:55+00:00,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,...,18731.0,11500.834,6499.657,0.0,730.509,81339.650,45968.825,0,5166.525,1.769452
2020-11-04 18:34:58+00:00,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,...,39556.0,24880.724,13053.480,0.0,1621.796,108207.499,56770.230,0,7053.271,1.906061
2020-11-09 19:09:13+00:00,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,...,18939.0,11780.058,6363.504,0.0,795.438,118783.340,64165.920,0,8020.740,1.851190
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-11-11 20:35:54+00:00,9846,276630,99,edison,wyoming,276765,174419,249061,255849,0.700,...,6.0,4.200,1.596,0.0,0.204,193641.000,73583.580,0,9405.420,2.631579
2020-11-11 20:36:41+00:00,9847,276648,99,edison,wyoming,276765,174419,249061,255849,0.700,...,18.0,12.600,4.788,0.0,0.612,193653.600,73588.368,0,9406.032,2.631579
2020-11-11 20:37:31+00:00,9848,276679,99,edison,wyoming,276765,174419,249061,255849,0.700,...,31.0,21.700,8.246,0.0,1.054,193675.300,73596.614,0,9407.086,2.631579


In [154]:
# Need to find the growing increments of votes for each time by each state
filt = (dfn['state'] == 'wisconsin')
dfs = dfn[filt]
dfs[['votes','votes_diff','trump_v','biden_v','r_to_d']]

,votes,votes_diff,trump_v,biden_v,r_to_d
timestamp,,,,,
2020-11-04 10:02:04+00:00,0,0.0,0.000,0.000,0.000000
2020-11-04 02:14:29+00:00,1095,1095.0,280.320,793.875,0.353103
2020-11-04 02:18:58+00:00,9001,7906.0,4893.814,2861.972,1.709945
2020-11-04 02:23:05+00:00,43641,34640.0,12435.760,21615.360,0.575321
2020-11-04 02:24:58+00:00,109676,66035.0,35857.005,29121.435,1.231293
2020-11-04 02:27:50+00:00,116441,6765.0,3720.750,2929.245,1.270208
2020-11-04 02:27:56+00:00,138795,22354.0,11087.584,10886.398,1.018480
2020-11-04 02:28:29+00:00,139671,876.0,436.248,425.736,1.024691
2020-11-04 02:29:02+00:00,213227,73556.0,34865.544,37366.448,0.933071


In [153]:
# fig = go.Figure()
# # fig.add_trace(go.Scatter(x=dfs.index,y=[dfs['vote_share_dem'],dfs['vote_share_rep']]))
# fig.add_trace(go.Scatter(x=dfs.index,y=dfs['vote_share_dem'],name='Dem'))
# fig.add_trace(go.Scatter(x=dfs.index, y=dfs['vote_share_rep'],name='Rep'))

fig = px.line(dfs,x=dfs.index,y=[dfs['vote_share_dem'],dfs['vote_share_rep']])

fig.show()

In [143]:
sortedstates = dfn.groupby('state')['r_to_d'].last().to_frame(name='r_to_d')
sortedstates.sort_values(by='r_to_d', ascending = False, inplace = True)
sortedstates.index

Index(['wyoming', 'west-virginia', 'north-dakota', 'oklahoma', 'idaho',
       'arkansas', 'south-dakota', 'kentucky', 'alabama', 'tennessee', 'utah',
       'nebraska', 'louisiana', 'mississippi', 'montana', 'indiana',
       'missouri', 'kansas', 'south-carolina', 'alaska', 'iowa', 'ohio',
       'texas', 'florida', 'north-carolina', 'georgia', 'arizona', 'wisconsin',
       'pennsylvania', 'nevada', 'michigan', 'minnesota', 'new-hampshire',
       'maine', 'virginia', 'new-mexico', 'colorado', 'new-york', 'new-jersey',
       'oregon', 'illinois', 'delaware', 'washington', 'connecticut',
       'rhode-island', 'hawaii', 'california', 'maryland', 'massachusetts',
       'vermont'],
      dtype='object', name='state')

In [144]:
# fig = go.Figure()
# fig.add_trace(go.Scatter(x=dfn.index,y=dfn['r_to_d'],name='Ratio',mode='markers',))
fig = px.line(dfn,x=dfn.index,y=dfn['r_to_d'],color=dfn['state'], category_orders={"state":sortedstates.index.tolist(), })
pio.write_html(fig, file=f'ratio.html', auto_open=True)
fig.show()

https://realpython.com/pandas-groupby/#:~:text=The%20%E2%80%9CHello%2C%20World!%E2%80%9D%20of%20Pandas%20GroupBy,-Now%20that%20you&text=You%20call%20.,a%20single%20column%20name%20to%20.

In [145]:
states = df['state'].unique()
filt = (df['state'] == 'alaska')
df[filt]

,Unnamed: 0,votes,eevp,eevp_source,state,expected_votes,trump2016,votes2012,votes2016,vote_share_rep,...,votes_diff,trump_v,biden_v,trd_v,check_v,trump_c,biden_c,trd_c,check_c,r_to_d
timestamp,,,,,,,,,,,,,,,,,,,,,
2020-11-04 10:01:44+00:00,0,0,0,edison,alaska,367000,163387,300495,318608,0.000,...,0.0,0.000,0.000,0.0,0.000,0.000,0.000,0,0.000,NaN
2020-11-04 07:47:41+00:00,1,113744,31,edison,alaska,367000,163387,300495,318608,0.605,...,113744.0,68815.120,40037.888,0.0,4890.992,68815.120,40037.888,0,4890.992,1.718750
2020-11-04 08:57:55+00:00,2,132475,36,edison,alaska,367000,163387,300495,318608,0.614,...,18731.0,11500.834,6499.657,0.0,730.509,81339.650,45968.825,0,5166.525,1.769452
2020-11-04 18:34:58+00:00,3,172031,56,edison,alaska,367000,163387,300495,318608,0.629,...,39556.0,24880.724,13053.480,0.0,1621.796,108207.499,56770.230,0,7053.271,1.906061
2020-11-09 19:09:13+00:00,4,190970,61,edison,alaska,367000,163387,300495,318608,0.622,...,18939.0,11780.058,6363.504,0.0,795.438,118783.340,64165.920,0,8020.740,1.851190
2020-11-11 02:46:15+00:00,5,240234,75,edison,alaska,367000,163387,300495,318608,0.580,...,49264.0,28573.120,18769.584,0.0,1921.296,139335.720,91529.154,0,9369.126,1.522310
2020-11-11 13:18:28+00:00,6,260983,75,edison,alaska,367000,163387,300495,318608,0.569,...,20749.0,11806.181,8112.859,0.0,829.960,148499.327,102044.353,0,10439.320,1.455243
2020-11-12 16:41:43+00:00,7,286788,78,edison,alaska,367000,163387,300495,318608,0.559,...,25805.0,14424.995,10399.415,0.0,980.590,160314.492,115575.564,0,10897.944,1.387097
2020-11-13 02:25:15+00:00,8,312033,99,edison,alaska,367000,163387,300495,318608,0.541,...,25245.0,13657.545,10628.145,0.0,959.310,168809.853,131365.893,0,11857.254,1.285036


In [146]:
fig = px.scatter_3d(df, x=df.index, y=df['state'], z=df['trump_c'],color=df['state'])
fig.show()
pio.write_html(fig, file=f'3d.html', auto_open=True)



In [147]:
# Need to find the growing increments of votes for each time by each state
pd.set_option('display.max_rows', 361)
filt = (df['state'] == 'wisconsin')
dfs = df[filt]
dfs[['votes','votes_diff','trump_v','biden_v','r_to_d']]

,votes,votes_diff,trump_v,biden_v,r_to_d
timestamp,,,,,
2020-11-04 10:02:04+00:00,0,0.0,0.000,0.000,NaN
2020-11-04 02:14:29+00:00,1095,1095.0,280.320,793.875,0.353103
2020-11-04 02:18:58+00:00,9001,7906.0,4893.814,2861.972,1.709945
2020-11-04 02:23:05+00:00,43641,34640.0,12435.760,21615.360,0.575321
2020-11-04 02:24:58+00:00,109676,66035.0,35857.005,29121.435,1.231293
2020-11-04 02:27:50+00:00,116441,6765.0,3720.750,2929.245,1.270208
2020-11-04 02:27:56+00:00,138795,22354.0,11087.584,10886.398,1.018480
2020-11-04 02:28:29+00:00,139671,876.0,436.248,425.736,1.024691
2020-11-04 02:29:02+00:00,213227,73556.0,34865.544,37366.448,0.933071


In [148]:

fig = go.Figure(data = [
    go.Scatter(x=dfs.index,y=dfs['biden_v'],name='Biden',mode='markers'),
    go.Scatter(x=dfs.index,y=dfs['trump_v'],name='Trump',mode='markers')
])
# fig = px.bar(x=dfs.index,y=[dfs['votes'],dfs['trump_v']])
pio.write_html(fig, file=f'graphs/bars.html', auto_open=True)
fig.show()